### __Option Pricing on S&P 500 Daily Risk Control 10% Index__

__The S&P 500 Daily Risk Control 10% Index (SPXT10UE) is part of S&P Dow Jones Risk Control Indices familly and use the below parameters__

| ***Index Name*** | ***Underlying Risk Index*** | ***Risk Control Level*** | ***Maximum Leverage*** | ***Interest Rate*** | ***Volatility Calculation*** | ***Return Frequency for Volatility*** | ***Lag to Rebalancing Date*** | ***Decay Factor (Short-Term)*** | ***Decay Factor (Long-Term)*** | ***Rebalancing Frequency*** | ***Launch Date*** | ***Bloomberg Tickers***                                                           |
|------------------------------------------------|------------------------------------------------------|------------------------|----------------------|-------------------------|----------------------------|--------------------------------------|-----------------------------|-------------------------------|------------------------------|---------------------------|---------------|---------------------------------------------------------------------------------|
| S&P 500 Daily Risk Control 10% Index|S&P 500 Total Return: SPTR (USD) | 10% | 150%                 | SOFR + 0.02963*         | Exponentially weighted    | Daily                                | 2 days                      | 94%                           | 97%                          | Daily                     | 10-Sep-09     | ***Excess Return:*** SPXT5UE (USD)|


__The S&P 500 Daily Risk Control indices are computed using the below methodology__

$$
\text{Risk Control ER Index Value}_t 
= 
\text{RiskControlERIndexValue}_{rb}
\,\times\,
\Biggl[
1
\;+\;
K_{rb}\,\biggl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-1}} \;-\; 1\biggr)
\;-\;
K_{rb}\,\Bigl(
  \prod_{i=rb+1}^{t}\bigl(1 + \text{InterestRate}_{i-1} \times \frac{D_{i-1,i}}{360}\bigr)
  \;-\; 1
\Bigr)
\Biggr]
$$


$$
K_{rb} 
= 
\min\!\Bigl(\text{Max }K,\;\frac{\text{Target Volatility}}{\text{Realized Volatility}_{rb-d}}\Bigr)
$$


$$
\text{RealizedVolatility}_t 
  = \max\bigl(\text{RealizedVolatility}_{S,t}, \text{RealizedVolatility}_{L,t}\bigr)
$$

$$
\text{RealizedVolatility}_{S,t} 
  = \sqrt{\frac{252}{n}\,\text{Variance}_{S,t}}
$$

$$
\text{RealizedVolatility}_{L,t}
  = \sqrt{\frac{252}{n}\,\text{Variance}_{L,t}}
$$

$$
\text{Variance}_{S,t}
  = \lambda_S\,\text{Variance}_{S,t-1}
   + \bigl(1 - \lambda_S\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

$$
\text{Variance}_{L,t}
  = \lambda_L\,\text{Variance}_{L,t-1}
   + \bigl(1 - \lambda_L\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

In [11]:
# -------
# IMPORT
# -------
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.express as px
from python_module.pricing_model import BlackScholesModel, SABRModel

In [ ]:
def compute_option(
    option_type, strike, option_time_to_maturity, r,
    convergence_time, lambda_s, lambda_l, target_volatility, max_k, lag,
    alpha, rho, nu):

    # -------
    # GENERATE ASSET 0 MONTECARLO
    # -------
    n_steps = option_time_to_maturity+convergence_time
    T = n_steps/250
    forward_process, vol_process = SABRModel.compute_montecarlo(F=1, T=T, alpha=alpha, beta=1, rho=rho, nu=nu, n_steps=n_steps, n_paths=10_000, seed=True)

    # -------
    # COMPUTE LONG AND SHORT VARIANCE
    # -------
    risk_control_montecarlo = dict()
    for path in forward_process:
        
        forward_price = forward_process[path]
        forward_price.name = 'forward'
        spot_var = np.log(forward_price / forward_price.shift()).pow(2).dropna()

        variance_s = list()
        variance_l = list()
        for i in range(spot_var.shape[0]):

            if len(variance_s) == 0 and len(variance_l) == 0:
                variance_s.append(spot_var.iloc[0])
                variance_l.append(spot_var.iloc[0])
                
            else:
                variance_s_value = lambda_s * variance_s[-1] + (1 - lambda_s) * spot_var.loc[i]
                variance_s.append(variance_s_value)

                variance_l_value = lambda_l * variance_l[-1] + (1 - lambda_l) * spot_var.loc[i]
                variance_l.append(variance_l_value)

    df = spot_var.to_frame('daily_var')
    df['variance_s'] = variance_s
    df['variance_l'] = variance_l
    df['realized_vol_s'] = np.sqrt(df['variance_s']*252)
    df['realized_vol_l'] = np.sqrt(df['variance_l']*252)
    df['realized_vol'] = df[['realized_vol_s', 'realized_vol_l']].max(axis=1)
    df = pd.concat([forward_price, df], axis=1)

    # -------
    # COMPUTE VOLATILITY TARGET
    # -------
    df['leverage'] = (target_volatility / df['realized_vol']).clip(upper=max_k)
    df['leverage_shifted'] = df['leverage'].shift(lag)
    df_ = df.iloc[-250:].copy()
    df_['daily_risk_control'] = (df_['forward'].pct_change() * df_['leverage_shifted']).fillna(0).add(1).cumprod()
    risk_control_montecarlo[path] = df_['daily_risk_control'].to_dict()
    risk_control_montecarlo = pd.DataFrame(risk_control_montecarlo).reset_index(drop=True)

    # -------
    # COMPUTE OPTION PRICE
    # -------
    if option_type == 'call':
        option_price = (risk_control_montecarlo.iloc[-1]-strike).clip(lower=0).mean()
    else:
        option_price = (strike-risk_control_montecarlo.iloc[-1]).clip(lower=0).mean()
    return option_price * np.exp(-r*T)


In [16]:
# -------
# INPUT
# -------

# Option contract sepcifications
option_type = 'call'
strike = 100
option_time_to_maturity = 250

# Volatility target settings
convergence_time = 250
lambda_s = 0.94
lambda_l = 0.97
target_volatility = 0.1
max_k = 1.5
lag = 2

# Model settings
alpha = 0.1
rho = 0.5
nu = 0.5

In [ ]:
# -------
# GENERATE ASSET 0 MONTECARLO
# -------
n_steps = option_time_to_maturity+convergence_time
T = n_steps/250
forward_process, vol_process = SABRModel.compute_montecarlo(F=1, T=T, alpha=alpha, beta=1, rho=rho, nu=nu, n_steps=n_steps, n_paths=10_000, seed=True)

In [14]:
risk_control_montecarlo = dict()
for path in forward_process:
    
    forward_price = forward_process[path]
    forward_price.name = 'forward'
    spot_var = np.log(forward_price / forward_price.shift()).pow(2).dropna()

    variance_s = list()
    variance_l = list()
    for i in range(spot_var.shape[0]):

        if len(variance_s) == 0 and len(variance_l) == 0:
            variance_s.append(spot_var.iloc[0])
            variance_l.append(spot_var.iloc[0])
            
        else:
            variance_s_value = lambda_s * variance_s[-1] + (1 - lambda_s) * spot_var.loc[i]
            variance_s.append(variance_s_value)

            variance_l_value = lambda_l * variance_l[-1] + (1 - lambda_l) * spot_var.loc[i]
            variance_l.append(variance_l_value)

    df = spot_var.to_frame('daily_var')
    df['variance_s'] = variance_s
    df['variance_l'] = variance_l
    df['realized_vol_s'] = np.sqrt(df['variance_s']*252)
    df['realized_vol_l'] = np.sqrt(df['variance_l']*252)
    df['realized_vol'] = df[['realized_vol_s', 'realized_vol_l']].max(axis=1)
    df = pd.concat([forward_price, df], axis=1)
    df['leverage'] = (target_volatility / df['realized_vol']).clip(upper=max_k)
    df['leverage_shifted'] = df['leverage'].shift(lag)
    df_ = df.iloc[-250:].copy()
    df_['daily_risk_control'] = (df_['forward'].pct_change() * df_['leverage_shifted']).fillna(0).add(1).cumprod()
    risk_control_montecarlo[path] = df_['daily_risk_control'].to_dict()
risk_control_montecarlo = pd.DataFrame(risk_control_montecarlo).reset_index(drop=True)

In [15]:
call_price = (risk_control_montecarlo.iloc[-1]-1).clip(lower=0).mean()
print(f'CALL PRICE: {call_price*100:.2F}%')

CALL PRICE: 3.44%
